In [1]:
import fastplotlib as fpl
import napari

In [2]:
from napari.layers import Image
from fastplotlib.graphics import ImageGraphic
from fastplotlib.graphics._base import Graphic

In [3]:
import numpy as np

In [4]:
data = np.random.rand(512, 512)

In [5]:
img = Image(data=data)

In [6]:
img.contrast_limits

[2.427121624237749e-06, 0.9999977274652415]

In [7]:
img.colormap.name

'gray'

In [8]:
def image_wrapper(image: Image) -> ImageGraphic:
    data = image.data
    name = image.name
    metadata = image.metadata
    cmap = image.colormap.name
    vmin = image.contrast_limits[0]
    vmax = image.contrast_limits[1]

    image_graphic = ImageGraphic(data=data, 
                        name=name, 
                        metadata=metadata,
                        cmap=cmap,
                        vmin=vmin,
                        vmax=vmax
                       )
    def update_vmin_vmax(ev):
        #print(dir(ev))
        image_graphic.cmap.vmin, image_graphic.cmap.vmax = ev.source.contrast_limits
        
    img.events.contrast_limits.connect(update_vmin_vmax)
    
    return image_graphic

In [9]:
plot2 = fpl.Plot()

RFBOutputContext()

In [10]:
img2 = image_wrapper(image=img)

/home/clewis7/repos/fastplotlib/fastplotlib/graphics/_features/_base.py:34: UserWarning: converting float64 array to float32
  warn(f"converting {array.dtype} array to float32")


In [11]:
plot2.add_graphic(img2)

In [12]:
plot2.show()

In [13]:
img.contrast_limits = (0, 0.5)

In [126]:
data = np.random.rand(12, 16, 18, 22, 24)

In [127]:
img = Image(data=data)

In [128]:
dims = napari.components.Dims()

In [129]:
img

<Image layer 'data' at 0x7fd8082c7050>

In [130]:
dims

Dims(ndim=2, ndisplay=2, order=(0, 1), axis_labels=('0', '1'), range=(RangeTuple(start=0.0, stop=2.0, step=1.0), RangeTuple(start=0.0, stop=2.0, step=1.0)), margin_left=(0.0, 0.0), margin_right=(0.0, 0.0), point=(0.0, 0.0), last_used=0)

In [131]:
from ipywidgets import IntSlider, VBox

In [132]:
from functools import partial

In [133]:
class FplViewer(napari.components.ViewerModel):
    plot: fpl.Plot = None
    sliders: list = None
    def __init__(
                self,
                *args,
                **kwargs):
        super().__init__(**kwargs)
        
        self.plot = fpl.Plot()
        
        self.sliders = list()
        self._connect_layer_events()
    
    def _add_graphic(self, ev):
        # parse graphic to add
        layer_insert_ix = ev.index
        print(self.dims)
        if type(self.layers[layer_insert_ix]).__name__ == "Image":
            print("adding image")
            layer = self.layers[layer_insert_ix]
            self.plot.add_image(
                data=layer._slice.image.view,
                name=layer.name, 
                metadata=layer.metadata,
                cmap=layer.colormap.name,
                vmin=layer.contrast_limits[0],
                vmax=layer.contrast_limits[1],
            )
            print(self.dims)
        else:
            raise NotImplementedError
        
        #self._update_sliders()
    
    def _remove_graphic(self, ev):
        layer_remove_ix = ev.index
        graphic = self.plot.graphics[layer_remove_ix]
        self.plot.remove_graphic(graphic)
        
    def _update_sliders(self):
        self.sliders = list()
        
        dims_not_displayed = list(range(self.dims.ndim))[:-2]
        
        for dim in dims_not_displayed[::-1]:
            slider = IntSlider(value=int(self.dims.point[dim]),
                              min=self.dims.range[dim].start,
                              max=self.dims.range[dim].stop,
                              step=self.dims.range[dim].step)
            
            slider.observe(partial(self._update_point, dim), "value")
            self.sliders.append(slider)
        
    def _update_point(self, dim, change):
        new_val = change["new"]
        self.dims.set_point(dim, new_val)
        for layer in self.layers:
            self.plot[layer.name].data = layer._slice.image.view
        
    
    def _connect_layer_events(self):
        self.layers.events.inserted.connect(self._add_graphic, position="last")
        self.layers.events.inserted.connect(self._update_sliders, position="last")
        self.layers.events.removed.connect(self._remove_graphic)
        self.layers.events.removed.connect(self._update_sliders)
    
    def _connect_dims_events(self):
        pass
    
    def show(self):
        return VBox([self.plot.show(),
                     *self.sliders]
                   )

In [134]:
fpl_view = FplViewer()

RFBOutputContext()

In [135]:
fpl_view.add_image(data)

ndim=5 ndisplay=2 order=(0, 1, 2, 3, 4) axis_labels=('0', '1', '2', '3', '4') range=(RangeTuple(start=0.0, stop=11.0, step=1.0), RangeTuple(start=0.0, stop=15.0, step=1.0), RangeTuple(start=0.0, stop=17.0, step=1.0), RangeTuple(start=0.0, stop=21.0, step=1.0), RangeTuple(start=0.0, stop=23.0, step=1.0)) margin_left=(0.0, 0.0, 0.0, 0.0, 0.0) margin_right=(0.0, 0.0, 0.0, 0.0, 0.0) point=(5.0, 7.0, 8.0, 10.0, 11.0) last_used=0
adding image
ndim=5 ndisplay=2 order=(0, 1, 2, 3, 4) axis_labels=('0', '1', '2', '3', '4') range=(RangeTuple(start=0.0, stop=11.0, step=1.0), RangeTuple(start=0.0, stop=15.0, step=1.0), RangeTuple(start=0.0, stop=17.0, step=1.0), RangeTuple(start=0.0, stop=21.0, step=1.0), RangeTuple(start=0.0, stop=23.0, step=1.0)) margin_left=(0.0, 0.0, 0.0, 0.0, 0.0) margin_right=(0.0, 0.0, 0.0, 0.0, 0.0) point=(5.0, 7.0, 8.0, 10.0, 11.0) last_used=0


/home/clewis7/repos/fastplotlib/fastplotlib/graphics/_features/_base.py:34: UserWarning: converting float64 array to float32
  warn(f"converting {array.dtype} array to float32")


<Image layer 'data' at 0x7fd8082c7250>

In [136]:
fpl_view.show()

In [83]:
fpl_view.layers.events.inserted.callbacks

((<weakref at 0x7fd80a9cc680; to 'FplViewer' at 0x7fd80aa83bd0>,
  '_add_graphic'),
 (<weakref at 0x7fd80a9cc680; to 'FplViewer' at 0x7fd80aa83bd0>,
  '_on_add_layer'))

In [79]:
data[::-2]

array([[[0.37185324, 0.80148295, 0.83298898, ..., 0.44304116,
         0.29954418, 0.01195732],
        [0.48310552, 0.86980409, 0.16189922, ..., 0.0700376 ,
         0.40806886, 0.43373978],
        [0.02139306, 0.48878722, 0.56990265, ..., 0.62712702,
         0.32751719, 0.76502095],
        ...,
        [0.72207004, 0.71520886, 0.33093651, ..., 0.12306302,
         0.05436172, 0.28599215],
        [0.3400389 , 0.643539  , 0.7033658 , ..., 0.35260615,
         0.39878405, 0.67037841],
        [0.08288548, 0.37085233, 0.30960618, ..., 0.84602274,
         0.15659999, 0.25223289]],

       [[0.44470302, 0.00833546, 0.00855313, ..., 0.77950297,
         0.92194599, 0.0542394 ],
        [0.39696722, 0.18842611, 0.4241916 , ..., 0.38837973,
         0.42978768, 0.28127854],
        [0.88591269, 0.9838238 , 0.13045192, ..., 0.76590735,
         0.37193117, 0.65982193],
        ...,
        [0.08875013, 0.29311482, 0.95130337, ..., 0.50610968,
         0.25004579, 0.92486655],
        [0.9